In [1]:
import tkinter as tk
from tkinter import ttk, messagebox

class SudokuSolver:
    def __init__(self, root):
        self.root = root
        self.root.title("Sudoku Solver with Difficulty Levels")
        self.create_widgets()
        self.grid = [[0] * 9 for _ in range(9)]
        self.puzzles = {
            'Easy': [
                # An easy Sudoku puzzle
                "530070000600195000098000060800060003400803001700020006060000280000419005000080079"
            ],
            'Medium': [
                # A medium Sudoku puzzle
                "600120384008459072000006005000264030070080006940003000310000050089700000502000190"
            ],
            'Hard': [
                # A hard Sudoku puzzle
                "000260701680070090190004500820100040004602900050003028009300074040050036703018000"
            ]
        }

    def create_widgets(self):
        self.entries = []
        frame = ttk.Frame(self.root, padding="5")
        frame.grid(row=0, column=0, sticky="nsew")

        for row in range(9):
            for col in range(9):
                entry_var = tk.StringVar()
                entry = ttk.Entry(frame, width=2, font=('Arial', 20), textvariable=entry_var, justify='center')
                entry.grid(row=row, column=col, sticky='nsew', padx=1, pady=1)
                if (row // 3, col // 3) in [(1, 0), (0, 1), (2, 1), (1, 2)]:
                    entry.config(background='light grey')
                entry_var.trace("w", lambda name, index, mode, sv=entry_var: self.entry_callback(sv))
                self.entries.append((entry, entry_var))

        ttk.Button(frame, text='Solve', command=self.solve_sudoku).grid(row=9, column=0, columnspan=5, sticky='nsew')
        ttk.Button(frame, text='Clear', command=self.clear).grid(row=9, column=5, columnspan=4, sticky='nsew')

        difficulty = ttk.Combobox(frame, values=['Easy', 'Medium', 'Hard'], state="readonly")
        difficulty.grid(row=10, column=0, columnspan=9, sticky='nsew')
        difficulty.bind('<<ComboboxSelected>>', self.load_puzzle)

    def entry_callback(self, sv):
        value = sv.get()
        if len(value) > 1 or (value and not value.isdigit()):
            sv.set('')

    def solve_sudoku(self):
        if not self.read_grid():
            messagebox.showerror("Error", "Invalid input.")
            return
        if self.backtracking_search():
            self.update_grid()
        else:
            messagebox.showinfo("Result", "No solution exists or puzzle is unsolvable!")

    def load_puzzle(self, event):
        difficulty = event.widget.get()
        puzzle = self.puzzles.get(difficulty)[0]
        self.set_grid(puzzle)

    def set_grid(self, puzzle):
        for i, char in enumerate(puzzle):
            row, col = divmod(i, 9)
            self.entries[row * 9 + col][1].set(char if char != '0' else '')

    def read_grid(self):
        for i, entry in enumerate(self.entries):
            value = entry[1].get()
            if value and (not value.isdigit() or not (1 <= int(value) <= 9)):
                return False
            self.grid[i // 9][i % 9] = int(value) if value.isdigit() else 0
        return True

    def update_grid(self):
        for i, entry in enumerate(self.entries):
            entry[1].set(str(self.grid[i // 9][i % 9]) if self.grid[i // 9][i % 9] != 0 else '')

    def backtracking_search(self):
        empty = self.find_unassigned_location()
        if not empty:
            return True
        row, col = empty
        for num in range(1, 10):
            if self.is_safe(num, row, col):
                self.grid[row][col] = num
                if self.backtracking_search():
                    return True
                self.grid[row][col] = 0
        return False

    def find_unassigned_location(self):
        for row in range(9):
            for col in range(9):
                if self.grid[row][col] == 0:
                    return (row, col)
        return None

    def is_safe(self, num, row, col):
        return self.is_row_safe(num, row) and \
               self.is_col_safe(num, col) and \
               self.is_box_safe(num, row - row % 3, col - col % 3)

    def is_row_safe(self, num, row):
        return all(num != self.grid[row][col] for col in range(9))

    def is_col_safe(self, num, col):
        return all(num != self.grid[row][col] for row in range(9))

    def is_box_safe(self, num, box_start_row, box_start_col):
        return all(num != self.grid[r][c]
                   for r in range(box_start_row, box_start_row + 3)
                   for c in range(box_start_col, box_start_col + 3))

    def clear(self):
        for entry in self.entries:
            entry[1].set('')

def main():
    root = tk.Tk()
    app = SudokuSolver(root)
    root.mainloop()

if __name__ == "__main__":
    main()